# NPL Natural Disasters

In [1]:
%%capture

import collections
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import spacy
# import scattertext as st

from unidecode import unidecode

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
)

from sklearn.model_selection import train_test_split, cross_val_score

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("punkt")
nltk.download("words")
nltk.download("stopwords")


!python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_trf


2022-12-03 18:46:43.258053: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 18:46:43.292477: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# load datasets

df_train = pd.read_csv("data/train.csv")
df_train["train"] = 1
df_test = pd.read_csv("data/test.csv")
df_test["train"] = 0
df = pd.concat([df_train, df_test], axis=0)
df


,id,keyword,location,text,target,train
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0,1
...,...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,NaN,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,NaN,0
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,NaN,0
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,NaN,0


## Data Cleaning

In [3]:
# perform data cleaning

# 1. convert all letters to lowercase, so capitalization does not influence word frequency
df["text_cleaned"] = df["text"].str.lower()

# 2. remove @mention
df["text_cleaned"] = df["text_cleaned"].str.replace("@\w+|.@\w+", "", regex=True)

# 3. remove emoji
# credit: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
df["text_cleaned"] = df["text_cleaned"].str.replace(
    "\U0001F600-\U0001F64F", "", regex=True
)
df["text_cleaned"] = df["text_cleaned"].str.replace(
    "\U0001F300-\U0001F5FF", "", regex=True
)
df["text_cleaned"] = df["text_cleaned"].str.replace(
    "\U0001F680-\U0001F6FF", "", regex=True
)
df["text_cleaned"] = df["text_cleaned"].str.replace(
    "\U0001F680-\U0001F6FF", "", regex=True
)

# 4. remove non alphabetical characters
df["text_cleaned"] = df["text_cleaned"].str.replace("/[^\x00-\x7F]+/", "", regex=True)

# 4. remove special character referencing starting with &
df["text_cleaned"] = df["text_cleaned"].str.replace("&[a-z]+;?", "", regex=True)

# 5. remove http and https links
df["text_cleaned"] = df["text_cleaned"].str.replace(
    "((?:http|https):\/\/[-\w\d+=&@#\/%?~|!:;\.,]*)", "", regex=True
)

# 6. remove punctuations with a space, except for apostrophes
# note that many text do not have space after punctuations, to prevent joining separate words, replace with space
df["text_cleaned"] = df["text_cleaned"].str.replace("[^\w\s']", " ", regex=True)

# 7. translate non ascii text to closest representation
# credit: https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string
df["text_cleaned"] = df["text_cleaned"].apply(lambda x: unidecode(x))

# 8. remove blocks of numbers
df["text_cleaned"] = df["text_cleaned"].str.replace("\d+", " ", regex=True)

# 9. remove extra white spaces before after sentences, as well as between words
df["text_cleaned"] = df["text_cleaned"].str.replace("\s+", " ", regex=True)

# 10. remove misc bad letters or invalid letter blocks
df["text_cleaned"] = df["text_cleaned"].str.replace("u_", " ")

# 11. finally stripe all leading and ending whitespaces and numbers
df["text_cleaned"] = df["text_cleaned"].str.strip("123.!? \n\t")


After the above 9 simple data cleaning, most of our tweets are free of invalid characters and text strings. There are still two issues remain:

* **misspelling of words**: a number of words provided in the dataset had been misspelled. Without correcting word spelling, the misspelled words will likely be either discarded during feature engineering and selection, and have little predictive power if included.
* **retweets and nearly identical tweets**: identical tweets and retweets can bias the dataset toward a specific set of words and phrases. It's better if identical copy or very similar tweets are removed.


In [4]:
# # To fix mispelling of words, we cab use jaccard_distance.
# def sentenceSpellCorrect(sentence):

#     # credit: https://www.geeksforgeeks.org/correcting-words-using-nltk-in-python/
#     # list of incorrect spellings
#     # that need to be corrected
#     check_words = sentence.split(" ")
#     new_words = []

#     # loop for finding correct spellings
#     # based on jaccard distance
#     # and printing the correct word
#     for word in check_words:
#         try:
#             temp = [
#                 (jaccard_distance(set(ngrams(word, 2)), set(ngrams(w, 2))), w)
#                 for w in correct_words
#                 if w[0] == word[0]
#             ]
#             new = sorted(temp, key=lambda val: val[0])[0][1]
#             if word != new:
#                 print(f"{word} has been corrected to {new}")
#             new_words.append(new)

#         except:
#             print(f"error occurred when processing sentence: '{sentence}'")
#     new_sentence = " ".join(new_words)
#     return new_sentence


# global correct_words
# correct_words = words.words()
# df["text_cleaned"] = df["text_cleaned"].apply(lambda x: sentenceSpellCorrect(x))


at the end of data cleaning, separate test and train datasets


In [5]:
df_train = df[df.train == 1]
df_train.head(5)


,id,keyword,location,text,target,train,text_cleaned
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0,1,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0,1,all residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0,1,people receive wildfires evacuation orders in ...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0,1,just got sent this photo from ruby alaska as s...


In [6]:
df_test = df[df.train == 0]
df_test.head(5)


,id,keyword,location,text,target,train,text_cleaned
0,0,NaN,NaN,Just happened a terrible car crash,NaN,0,just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",NaN,0,heard about earthquake is different cities sta...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",NaN,0,there is a forest fire at spot pond geese are ...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,NaN,0,apocalypse lighting spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,NaN,0,typhoon soudelor kills in china and taiwan


In [7]:
# for the rest of the project, we will no longer be using df_test, so set df_train to df
df = df_train


## Task 1: Bag of words model

A bag of words model is a simple method of processing text data where a string of text (document) is converted to an array of words. These words are then insert into a collection with no specific order. A count of the words by category can reveal what type of words are most frequently occurring by category.

The bag of words is also known as unigrams and can be used as features into predictive models for classification and clustering. In our exercise, we ran through the full set of unigrams generated with a simple feature selection for low sample size (1 occurrence only) and with a variance threshold of 0.005. This removes a number of features that provide little information and differentiation in our model, and prevents overfitting.

Finally, a simple logistic regression model is used to model the label for disaster tweets. The default `L2` penalty term is used so we can use the `lbfgs` solver for faster model convergence.

### 1A. Build bag of words / unigrams (Feature Engineering)

In [8]:
# iterate through the entire training df and build bag of words (tokens) for target == 1 and target == 0

bag_target_0 = []
for sentence in list(df[df.target == 0]["text_cleaned"]):
    tokens = word_tokenize(sentence)
    bag_target_0 = bag_target_0 + tokens

bag_target_1 = []
for sentence in list(df[df.target == 1]["text_cleaned"]):
    tokens = word_tokenize(sentence)
    bag_target_1 = bag_target_1 + tokens


In [9]:
# perform preprocessing to further clean the word list

# three main preprocessing are done on tokens or word list
# 1. remove non alphanumeric characters or symbols that slip through data cleaning
# 2. remove any stop words like the, a, etc. that are present for grammatical reasons and are semantically meaningful
# 3. convert similar words together via Lemmatization, such that the base word is used
wordnet = WordNetLemmatizer()

bag_target_0_processed = bag_target_0
bag_target_0_processed = [i for i in bag_target_0_processed if i.isalpha()]
bag_target_0_processed = [
    i for i in bag_target_0_processed if i not in stopwords.words("english")
]
bag_target_0_processed = [wordnet.lemmatize(i) for i in bag_target_0_processed]
count_bag_target_0_processed = collections.Counter(bag_target_0_processed)
# count_bag_target_0_processed.most_common(10)

bag_target_1_processed = bag_target_1
bag_target_1_processed = [i for i in bag_target_1_processed if i.isalpha()]
bag_target_1_processed = [
    i for i in bag_target_1_processed if i not in stopwords.words("english")
]
bag_target_1_processed = [wordnet.lemmatize(i) for i in bag_target_1_processed]
count_bag_target_1_processed = collections.Counter(bag_target_1_processed)
# count_bag_target_1_processed.most_common(10)


In [10]:
# compute frequency of bag of words

count_bag_target_0_processed = dict(
    zip(count_bag_target_0_processed.keys(), count_bag_target_0_processed.values())
)
count_bag_target_1_processed = dict(
    zip(count_bag_target_1_processed.keys(), count_bag_target_1_processed.values())
)

df_bag_target_0 = pd.DataFrame(
    list(count_bag_target_0_processed.items()), columns=["word", "count"]
)
df_bag_target_1 = pd.DataFrame(
    list(count_bag_target_1_processed.items()), columns=["word", "count"]
)

df_bag_target_0["freq"] = df_bag_target_0["count"] / df_bag_target_0["count"].sum()
df_bag_target_1["freq"] = df_bag_target_1["count"] / df_bag_target_1["count"].sum()

df_bag_target_0 = df_bag_target_0.sort_values(by="count", ascending=False)
df_bag_target_0 = df_bag_target_0.reset_index(drop=True)
df_bag_target_1 = df_bag_target_1.sort_values(by="count", ascending=False)
df_bag_target_1 = df_bag_target_1.reset_index(drop=True)


review word count of 10 most common words for non-disaster tweets

In [11]:
df_bag_target_0.head(10)


,word,count,freq
0,like,256,0.007152
1,u,214,0.005978
2,get,184,0.005140
3,new,170,0.004749
4,one,138,0.003855
5,body,117,0.003269
6,time,106,0.002961
7,would,105,0.002933
8,day,104,0.002905
9,video,102,0.002849


review word count of 10 most common words for disaster tweets

In [12]:
df_bag_target_1.head(10)


,word,count,freq
0,fire,266,0.008747
1,u,162,0.005327
2,news,148,0.004867
3,disaster,122,0.004012
4,via,121,0.003979
5,year,112,0.003683
6,california,111,0.003650
7,police,109,0.003584
8,suicide,106,0.003486
9,family,105,0.003453


#### Summary of BOW results

Of the top 10 most common words, we see more words related to disasters like *fire*, *disaster*, *police*, etc in the disaster bag, and more neutral and non disaster words like *get*, *time*, and *would*. From the common words, we can also identify words that are common in both but is generally not very meaningful like *u*, *year*, but occurs frequently. These common words in both bags can introduce confusion for a classification model on predicting disaster tweets.

### 1B. Feature selection

Remove low sample rate features

In [13]:
# to start, remove low occurrence words
# these words occurs so rarely that they have little predictive power

bag_words = bag_target_0_processed + bag_target_1_processed
count_bag_words = collections.Counter(bag_words)

count_bag_words = dict(zip(count_bag_words.keys(), count_bag_words.values()))
df_bag_words = pd.DataFrame(list(count_bag_words.items()), columns=["word", "count"])
LOW_OCCURRENCE_THRESHOLD = 1
df_bag_words = df_bag_words[df_bag_words["count"] > LOW_OCCURRENCE_THRESHOLD]


In [14]:
# build training dataset using bag of words
def buildFeaturesFromWords(df, df_text_col, word_list):
    df = df.copy()
    series_list = []
    for word in word_list:
        # count number of this word in the sentence text
        if len(word) > 1:
            # discard single letter words
            s = df[df_text_col].str.count(word).rename(f"_{word}")
            series_list.append(s)
    return pd.concat([df, pd.concat(series_list, axis=1)], axis=1)


unique_words = list(df_bag_words.word.unique())
# make sure the unique word list is smaller than separate bags
assert (len(df_bag_target_0) + len(df_bag_target_1)) > len(unique_words)

df = buildFeaturesFromWords(df=df, df_text_col="text_cleaned", word_list=unique_words)

# build a list of variable names that correspond to df columns
unigram_list = [f"_{i}" for i in unique_words if len(i) > 1]

df.head(5)


,id,keyword,location,text,target,train,text_cleaned,_man,_love,_fruit,...,_idfire,_scariest,_ekiti,_salvadoran,_exchanging,_exc,_fatally,_ushed,_kashmir,_woodlawn
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0,1,our deeds are the reason of this earthquake ma...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0,1,forest fire near la ronge sask canada,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0,1,all residents asked to 'shelter in place' are ...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0,1,people receive wildfires evacuation orders in ...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0,1,just got sent this photo from ruby alaska as s...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Remove variables based on variance threshold

In [15]:
pred_var = "target"
X, y = df[unigram_list], df_train[pred_var]


In [16]:
from sklearn.feature_selection import VarianceThreshold

LOW_VAR_THRESHOLD = 0.005
sel = VarianceThreshold(threshold=LOW_VAR_THRESHOLD)
sel.fit(X)
# print(sel.get_feature_names_out())
print(
    f"by applying a low variance threshold of {LOW_VAR_THRESHOLD}, {len(unigram_list) - len(sel.get_feature_names_out())} out of {len(unigram_list)} variables were removed"
)

selected_unigram_list = list(sel.get_feature_names_out())


by applying a low variance threshold of 0.005, 4668 out of 5626 variables were removed


In [ ]:
pred_var = "target"
X, y = df[selected_unigram_list], df_train[pred_var]


### 1C. Logistic regression model

In [ ]:
from sklearn import linear_model

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0, train_size=0.8
)
reg = linear_model.LogisticRegression(solver="lbfgs", max_iter=1000)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f"score: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["disaster", "normal"]))


In [ ]:
y_pred_within = reg.predict(X_train)
cm = confusion_matrix(y_train, y_pred_within)

print(f"score(within sample): {accuracy_score(y_train, y_pred_within)}")
print(classification_report(y_train, y_pred_within, target_names=["disaster", "normal"]))

print("Confusion matrix format:\n [['TP', 'FP'],\n ['FN', 'TN']]")
print("Confusion matrix for within sample predictions (percentage):")
print(cm / cm.sum() * 100)


In [ ]:
cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix for out of sample predictions")
print(cm / cm.sum() * 100)


The simple logistic regression model using bag of words features performed reasonably well. However, the confusion matrix for out of sample prediction is noticeably worse than within sample prediction. This can be an indication of overfitting.

Additional effort in using cross validation methods (k-fold), and regularization method (ridge classification) that is more robust to overfitting can reduce model overfitting.

In [ ]:
#  evaluate and plot the model

df.loc[df["target"] == 0, "target_name"] = "Normal"
df.loc[df["target"] == 1, "target_name"] = "Disaster"

# generate probability for all samples for plotting
y_pred_all = reg.predict_proba(X)
df["prob_1"] = y_pred_all[:, 1]


In [ ]:
# this code block has error

# from scattertext import CorpusFromPandas, produce_scattertext_explorer

# nlp = spacy.load("en_core_web_sm")
# corpus = st.CorpusFromPandas(
#     df.sample(1000), category_col="target_name", text_col="text_cleaned", nlp=nlp
# ).build()

# html = st.produce_scattertext_explorer(
#     corpus,
#     category="target_name",
#     category_name="Disaster",
#     not_category_name="Normal",
#     width_in_pixels=1000,
#     minimum_term_frequency=0,
#     transform=st.Scalers.scale,
#     # metadata=df['speaker']
# )
# file_name = "Conventions2012ScattertextScale.html"
# open(file_name, "wb").write(html.encode("utf-8"))
# IFrame(src=file_name, width=1200, height=700)


In [ ]:
df["text_short"] = df["text_cleaned"].str.slice(0, 6)


In [ ]:
fig = sns.stripplot(df, x="text_short", y="prob_1", hue="target_name", size=1.5)
# fig.set(xlabel=None)
fig.set(xticklabels=[])
fig.tick_params(bottom=False)
plt.axhline(y=0.5, color="red", linestyle="--")
plt.legend(loc="upper right")


There are still a number of misclassification with the simple Logistic Regression model with bag of words.

We hope to improve upon this method by accounting for the importance of words in the dataset using Term Frequency and Inverse Document Frequency (TF-IDF), in the next task.

### 1D. Summary for Log1-BOW Model (Needs update)

Precision (TP / [TP + FP]) indicates the fraction of items labelled true in `target` that is actually true. In the **Log2-IDIEF** model, the positive predictive value of `normal` is *slightly higher* than `disaster`, indicating that the model has a bias in predicting normal. Generally, a higher precision could be concerning if we do not have resources to respond to every disaster predicted.

Recall (TP / [TP + FN]), indicates the fraction of item labelled true that was originally true. The sensitivity of the model at predicting disaster is *much higher* than normal category. This indicate disaster prediction is unlikely to be a false negative. Generally, a higher recall is good if we do not want to miss any real disaster event tweets.

In the confusion table below, we can see that the prediction for False Positive (FP) is *much lower* than False Negative (FN), and the confusion matrix values for FP and FN are both higher in the out of sample prediction. This is consistent with the precision and recall values we see in the classification report.

Unfortunately, ngrams with unigrams and bigrams performed worse than the simple unigram model, model **Log1-BOW**. This could be due to the fact that most bigrams don't add enough useful information to each feature to justify the larger feature space, resulting in additional noise in the model.

## Task 2: Feature generation and traditional ML model with TF-IDF

In [ ]:
# clean up dataframe to have the cleaned text, target, and full list of unigram
df = df[["id", "keyword", "location", "text_cleaned", "target"]]
df.shape


### 2A. Build ngrams with TF-IDF (Feature Engineering)

In addition to the issue of weighting with bag of words, the use of unigram also means the feature lacks context. For Task 2 with TF-IDF, in addition to unigram, bigrams are generated as a set of features to enhance the ability of models to capture context.

For task 2, the TfidfVectorizer from sklearn is used to generate both ngrams and the TF-IDF score.

_About TF-IDF Score_

TF = (Count of this word in the document) / (Total number of words in the document)
IDF = log {(Total number of document) / [(Number of document containing the word) + 1]}

TF-IDF = TF * IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    ngram_range=(1, 2),  # unigrams and bigrams
    stop_words="english",
)
t0 = time()
X_tfidf = vectorizer.fit_transform(df.text_cleaned)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")


In [ ]:
vectorizer.get_feature_names_out()


To prevent overfitting, we reduce the number of features by setting a higher low variance threshold for bigrams.

In [ ]:
# perform feature selection

from sklearn.feature_selection import VarianceThreshold

LOW_VAR_THRESHOLD = 0.0001
sel = VarianceThreshold(threshold=LOW_VAR_THRESHOLD)
sel.fit(X_tfidf)
X_tfidf = sel.transform(X_tfidf)
# print(sel.get_feature_names_out())
print(
    f"by applying a low variance threshold of {LOW_VAR_THRESHOLD}, {len(vectorizer.get_feature_names_out()) - len(sel.get_feature_names_out())} out of {len(vectorizer.get_feature_names_out())} ngram variables were removed"
)

selected_ngram_list = list(sel.get_feature_names_out())

print(f"length of selected_ngram_list: {len(selected_ngram_list)}")


### 2B. Logistic regression model

In [ ]:
y = df_train[pred_var]

X_tfidf.shape


In [ ]:
from sklearn import linear_model

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, random_state=0, train_size=0.8
)
reg = linear_model.LogisticRegression(solver="lbfgs", max_iter=1000)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f"score: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["disaster", "normal"]))


In [ ]:
y_pred_within = reg.predict(X_train)
cm = confusion_matrix(y_train, y_pred_within)

print(f"score(within sample): {accuracy_score(y_train, y_pred_within)}")
print(classification_report(y_train, y_pred_within, target_names=["disaster", "normal"]))

print("Confusion matrix format:\n [['TP', 'FP'],\n ['FN', 'TN']]")
print("Confusion matrix for within sample predictions (percentage):")
print(cm / cm.sum() * 100)


In [ ]:
cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix for out of sample predictions")
print(cm / cm.sum() * 100)


In [ ]:
#  evaluate and plot the model

df.loc[df["target"] == 0, "target_name"] = "Normal"
df.loc[df["target"] == 1, "target_name"] = "Disaster"

# generate probability for all samples for plotting
y_pred_all = reg.predict_proba(X_tfidf)
df["prob_1"] = y_pred_all[:, 1]


fig = sns.stripplot(df, x="text_cleaned", y="prob_1", hue="target_name", size=1.5)
# fig.set(xlabel=None)
fig.set(xticklabels=[])
fig.tick_params(bottom=False)
plt.axhline(y=0.5, color="red", linestyle="--")
plt.legend(loc="upper right")


### 2C. Summary for Log1-IDIEF Model (Needs update)

Precision (TP / [TP + FP]) indicates the fraction of items labelled true in `target` that is actually true. In the **Log2-IDIEF** model, the positive predictive value of `normal` is *slightly higher* than `disaster`, indicating that the model has a bias in predicting normal. Generally, a higher precision could be concerning if we do not have resources to respond to every disaster predicted.

Recall (TP / [TP + FN]), indicates the fraction of item labelled true that was originally true. The sensitivity of the model at predicting disaster is *much higher* than normal category. This indicate disaster prediction is unlikely to be a false negative. Generally, a higher recall is good if we do not want to miss any real disaster event tweets.

In the confusion table below, we can see that the prediction for False Positive (FP) is *much lower* than False Negative (FN), and the confusion matrix values for FP and FN are both higher in the out of sample prediction. This is consistent with the precision and recall values we see in the classification report.

Unfortunately, ngrams with unigrams and bigrams performed worse than the simple unigram model, model **Log1-BOW**. This could be due to the fact that most bigrams don't add enough useful information to each feature to justify the larger feature space, resulting in additional noise in the model.

### 2D. Repeat experiment with only Unigram and TF-IDF, and logistic regression

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    ngram_range=(1, 1),  # unigrams only
    stop_words="english",
)
t0 = time()
X_tfidf = vectorizer.fit_transform(df.text_cleaned)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")


In [ ]:
from sklearn import linear_model

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, random_state=0, train_size=0.8
)
reg = linear_model.LogisticRegression(solver="lbfgs", max_iter=1000)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f"score: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["disaster", "normal"]))


In [ ]:
y_pred_within = reg.predict(X_train)
cm = confusion_matrix(y_train, y_pred_within)

print(f"score(within sample): {accuracy_score(y_train, y_pred_within)}")
print(classification_report(y_train, y_pred_within, target_names=["disaster", "normal"]))

print("Confusion matrix format:\n [['TP', 'FP'],\n ['FN', 'TN']]")
print("Confusion matrix for within sample predictions (percentage):")
print(cm / cm.sum() * 100)


In [ ]:
cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix for out of sample predictions")
print(cm / cm.sum() * 100)


In [ ]:
#  evaluate and plot the model

df.loc[df["target"] == 0, "target_name"] = "Normal"
df.loc[df["target"] == 1, "target_name"] = "Disaster"

# generate probability for all samples for plotting
y_pred_all = reg.predict_proba(X_tfidf)
df["prob_1"] = y_pred_all[:, 1]


fig = sns.stripplot(df, x="text_cleaned", y="prob_1", hue="target_name", size=1.5)
# fig.set(xlabel=None)
fig.set(xticklabels=[])
fig.tick_params(bottom=False)
plt.axhline(y=0.5, color="red", linestyle="--")
plt.legend(loc="upper right")


### 2E. Summary for Log1-IDIEF Model

Precision (TP / [TP + FP]) indicates the fraction of items labelled true in `target` that is actually true. In the **Log1-IDIEF** model, the positive predictive value of `normal` is *slightly higher* than `disaster`, indicating that the model has a bias in predicting normal. Generally, a higher precision could be concerning if we do not have resources to respond to every disaster predicted.

Recall (TP / [TP + FN]), indicates the fraction of item labelled true that was originally true. The sensitivity of the model at predicting disaster is *much higher* than normal category. This indicate disaster prediction is unlikely to be a false negative. Generally, a higher recall is good if we do not want to miss any real disaster event tweets.

In the confusion table below, we can see that the prediction for False Positive (FP) is *much lower* than False Negative (FN), and the confusion matrix values for FP and FN are both higher in the out of sample prediction. This is consistent with the precision and recall values we see in the classification report.

Overall, the **Log1-IDIEF** model with unigram and tf-idf actually performed very similarly to the previous **Log2-IDIEF** model.


## Task 3: Pre-trained word embeddings + linear classifier model

### 3A. Apply pre-trained GloVe embeddings to generate features

In [ ]:
%%capture

import tensorflow as tf
from tensorflow import keras

# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

# credit to example: https://keras.io/examples/nlp/pretrained_word_embeddings/

In [ ]:
df["rand"] = np.random.rand(len(df))


In [ ]:
train_words = df[df.rand > 0.2].text_cleaned
test_words = df[df.rand <= 0.2].text_cleaned
train_labels = df[df.rand > 0.2].target
test_labels = df[df.rand <= 0.2].target


In [ ]:
train_words.shape


In [ ]:
# # use our cleaned text to build a word index - this generates lots of misses because sentences have not been converted into word strings
# word_index = dict(zip(list(df.text_cleaned), range(len(df))))


We need to perform text vectorization, so we can map our sequence of words as tensors.

In [ ]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=50)
text_ds = tf.data.Dataset.from_tensor_slices(train_words).batch(128)
vectorizer.adapt(text_ds)

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

X_train = vectorizer(np.array([[s] for s in train_words])).numpy()
X_test = vectorizer(np.array([[s] for s in test_words])).numpy()
X_all = vectorizer(np.array([[s] for s in list(df.text_cleaned)])).numpy()

y_train = np.array(train_labels)
y_test = np.array(test_labels)

vectorizer.get_vocabulary()[-5:]


In [ ]:
# word index and voc length should be identical
assert len(voc) == len(word_index)


In [ ]:
# read glove word embeddings

path_to_glove_file = os.path.join("./data/glove.6B.100d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs


In [ ]:
embeddings_index["a"].shape


In [ ]:
# build an embedding matrix

num_tokens = len(word_index) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


In [ ]:
# use the prepared embedding matrix to create an embedding layer

from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)


In [ ]:
# apply glove text embedding

X_train = embedding_layer(X_train)
X_test = embedding_layer(X_test)
X_all = embedding_layer(X_all)

# y_train = np.array(train_labels)
# y_test = np.array(test_labels)


reshape 50 x 100 word embedding features into 5000

In [ ]:
X_train.shape


In [ ]:
X_train_old_i0 = X_train[0]
X_train_old_i0


In [ ]:
X_train = tf.reshape(X_train, shape=(len(X_train), 5000))
X_test = tf.reshape(X_test, shape=(len(X_test), 5000))
X_all = tf.reshape(X_all, shape=(len(X_all), 5000))
X_train[0]

In [ ]:
# check reshape is done correctly
(tf.reshape(X_train[0], (50, 100)) == X_train_old_i0)


In [ ]:
# We do not need to apply a full keras model

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten

# vocab_size = 20000
# max_length = 50
# model = Sequential()
# model.add(Embedding(vocab_size, 8, input_length=max_length))
# model.add(Flatten())
# model.add(Dense(1, activation="sigmoid"))
# model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# print(model.summary())


### 3B. Stochastic gradient descent with glove word embeddings (SGD-Glove)

In [ ]:
from sklearn import linear_model

reg = linear_model.SGDClassifier(
    loss="modified_huber", penalty="l2", max_iter=1000, tol=0.001
)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f"score: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["disaster", "normal"]))


In [ ]:
y_pred_within = reg.predict(X_train)
cm = confusion_matrix(y_train, y_pred_within)

print(f"score(within sample): {accuracy_score(y_train, y_pred_within)}")
print(classification_report(y_train, y_pred_within, target_names=["disaster", "normal"]))

print("Confusion matrix format:\n [['TP', 'FP'],\n ['FN', 'TN']]")
print("Confusion matrix for within sample predictions (percentage):")
print(cm / cm.sum() * 100)


In [ ]:
cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix for out of sample predictions")
print(cm / cm.sum() * 100)


In [ ]:
#  evaluate and plot the model

df.loc[df["target"] == 0, "target_name"] = "Normal"
df.loc[df["target"] == 1, "target_name"] = "Disaster"

# generate probability for all samples for plotting
y_pred_all = reg.predict_proba(X_all)
df["prob_1"] = y_pred_all[:, 1]


fig = sns.stripplot(df, x="text_cleaned", y="prob_1", hue="target_name", size=1.5)
# fig.set(xlabel=None)
fig.set(xticklabels=[])
fig.tick_params(bottom=False)
plt.axhline(y=0.5, color="red", linestyle="--")
plt.legend(loc="upper right")


### 3C. Summary for SGD-Glove model

Precision (TP / [TP + FP]) indicates the fraction of items labelled true in `target` that is actually true. In the **SGD-Glove** model, the positive predictive value of `normal` is *lower* than `disaster`, indicating that the model has a bias in predicting disaster.

Recall (TP / [TP + FN]), indicates the fraction of item labelled true that was originally true. The sensitivity of the model at predicting disaster and normal is fairly *similar*.

In the confusion table below, we can see that the prediction for False Positive (FP) is *slightly higher* than False Negative (FN), and the confusion matrix values for FP and FN are both *higher* in the out of sample prediction. This is consistent with the precision and recall values we see in the classification report.

Overall, the **SGD-Glove** model performed worse than the previous models. This is especially notable in the difference in accuracy and f1-score between the test and train sample. This is an indication of overfitting due to a large number of feature generated using the Glove pre-trained text embedding. Model more robust to overfitting like ensemble models (gradient boosted trees, and random forest) could be explored with the Glove text embedding in the future.


## Recommendation and Conclusion